# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9624253190>
├── 'a' --> tensor([[0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011]])
└── 'x' --> <FastTreeValue 0x7f96242532e0>
    └── 'c' --> tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                        [ 1.2313,  0.2176,  0.9506, -0.8789],
                        [ 1.2311,  1.0460, -0.3037, -0.9040]])

In [4]:
t.a

tensor([[0.6993, 0.6600, 0.6244],
        [1.2887, 1.0908, 1.2011]])

In [5]:
%timeit t.a

65.1 ns ± 0.0762 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9624253190>
├── 'a' --> tensor([[ 0.7487,  0.8315,  0.8024],
│                   [-0.8008, -1.4285, -1.8641]])
└── 'x' --> <FastTreeValue 0x7f96242532e0>
    └── 'c' --> tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                        [ 1.2313,  0.2176,  0.9506, -0.8789],
                        [ 1.2311,  1.0460, -0.3037, -0.9040]])

In [7]:
%timeit t.a = new_value

67.8 ns ± 0.0926 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.6993, 0.6600, 0.6244],
               [1.2887, 1.0908, 1.2011]]),
    x: Batch(
           c: tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040]]),
       ),
)

In [10]:
b.a

tensor([[0.6993, 0.6600, 0.6244],
        [1.2887, 1.0908, 1.2011]])

In [11]:
%timeit b.a

57.6 ns ± 0.0658 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-6.9926e-02,  1.0398e+00, -1.6261e-03],
               [-1.6765e+00,  2.3905e-01,  2.0615e+00]]),
    x: Batch(
           c: tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

817 ns ± 5.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 479 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f957fb655e0>
├── 'a' --> tensor([[[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]],
│           
│                   [[0.6993, 0.6600, 0.6244],
│                    [1.2887, 1.0908, 1.2011]]])
└── 'x' --> <FastTreeValue 0x7f957fb65340>
    └── 'c' --> tensor([[[-0.4750, -1.2156, -0.5794,  2.0314],
           

In [21]:
%timeit t_stack(trees)

31 µs ± 88.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9578f225e0>
├── 'a' --> tensor([[0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011],
│                   [0.6993, 0.6600, 0.6244],
│                   [1.2887, 1.0908, 1.2011]])
└── 'x' --> <FastTreeValue 0x7f9578f22220>
    └── 'c' --> tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                        [ 1.2313,  0.2176,  0.9506, -0.8789],
                        [ 1.2311,  1.0460, -0.3037, -0.9040],
   

In [23]:
%timeit t_cat(trees)

29.4 µs ± 54.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 91.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4750, -1.2156, -0.5794,  2.0314],
                       [ 1.2313,  0.2176,  0.9506, -0.8789],
                       [ 1.2311,  1.0460, -0.3037, -0.9040]],
              
                      [[-0.4750, -1.2156, -0.5794,  2.0314],
                       [ 1.2313,  0.2176,  0.9506, -0.8789],
                       [ 1.2311,  1.0460, -0.3037, -0.9040]],
              
                      [[-0.4750, -1.2156, -0.5794,  2.0314],
                       [ 1.2313,  0.2176,  0.9506, -0.8789],
                       [ 1.2311,  1.0460, -0.3037, -0.9040]],
              
                      [[-0.4750, -1.2156, -0.5794,  2.0314],
                       [ 1.2313,  0.2176,  0.9506, -0.8789],
                       [ 1.2311,  1.0460, -0.3037, -0.9040]],
              
                      [[-0.4750, -1.2156, -0.5794,  2.0314],
                       [ 1.2313,  0.2176,  0.9506, -0.8789],
                       [ 1.2311,  1.0460, -0.3037, -0.9040]],

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040],
                      [-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040],
                      [-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040],
                      [-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040],
                      [-0.4750, -1.2156, -0.5794,  2.0314],
                      [ 1.2313,  0.2176,  0.9506, -0.8789],
                      [ 1.2311,  1.0460, -0.3037, -0.9040],
                      [-0.4750, -1.2156, -0.5794,  2.0314],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 512 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 4.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
